# Streamflow streamflow gauges tables and shapefiles

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to the computation of the miscelaneous of streamflow indexes provided in this publication.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made avaialble in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* Geopandas=0.10.2
* Pandas
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* Daily streamflow measurements. 

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

# Import modules

In [1]:
import pandas as pd
import numpy as np
import datetime
import tqdm as tqdm
import os
import geopandas as gpd
from utils.general import *

# Configurations

In [2]:
# Only editable variable:
PATH = r"/Users/thiagomedeirosdonascimento/Library/CloudStorage/OneDrive-Personal/PhD/Eawag/Papers/Paper1_Database/Paper"

# Set some constants:
THRESHOLD_YR = 360
THRESHOLD_MO = 25
THRESHOLD_WE = 7

* #### The users should NOT change anything in the code below here. 

In [3]:
PATH_OUTPUT = "results/"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [4]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries

,id,area_km2,outlet_lat,outlet_lng,area_offic,area_calc,Code,basin_id,label_area,name,layer,path,geometry
0,DE01945,"144,000",50.937000,6.963000,144232,144432.885,DE01945,DE01945,0,BundespegelKoeln,None,None,"POLYGON Z ((7.96208 46.53708 0.00000, 7.96625 ..."
1,DE01946,"148,000",51.226000,6.770000,147680,147934.665,DE01946,DE01946,0,BundespegelDuesseldorf,None,None,"POLYGON Z ((7.96208 46.53708 0.00000, 7.96625 ..."
2,DE01947,"144,000",50.937000,6.963000,144232,144432.885,DE01947,DE01947,0,BundespegelKoeln,None,None,"POLYGON Z ((7.96208 46.53708 0.00000, 7.96625 ..."
3,DE01948,"148,000",51.226000,6.770000,147680,147934.665,DE01948,DE01948,0,BundespegelDuesseldorf,None,None,"POLYGON Z ((7.96208 46.53708 0.00000, 7.96625 ..."
4,DE01949,"159,000",51.757000,6.395000,159300,159352.653,DE01949,DE01949,0,BundespegelRees,None,None,"POLYGON Z ((7.96208 46.53708 0.00000, 7.96625 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15009,NL00015,148,51.812000,4.566000,None,148.000,NL00015,NL00015,888,Puttershoek,Catchment_Boundaries_NL_Merge,C:\Users\nascimth\Documents\Thiago\Eawag\Pytho...,"POLYGON Z ((4.49041 51.82958 0.00000, 4.49041 ..."
15010,NL00016,9500,51.350000,4.251000,None,9500.000,NL00016,NL00016,4,Schaar van Ouden Doel,Catchment_Boundaries_NL_Merge,C:\Users\nascimth\Documents\Thiago\Eawag\Pytho...,"POLYGON Z ((4.18958 51.29875 0.00000, 4.18958 ..."
15011,NL00017,21300,50.830000,5.697000,None,21300.000,NL00017,NL00017,4,Sint Pieter noord,Catchment_Boundaries_NL_Merge,C:\Users\nascimth\Documents\Thiago\Eawag\Pytho...,"POLYGON Z ((4.45041 49.91875 0.00000, 4.45041 ..."
15012,NL00014,3229,52.342353,6.103294,None,3229.000,NL00014,NL00014,6,Olst,Catchment_Boundaries_NL_Merge,C:\Users\nascimth\Documents\Thiago\Eawag\Pytho...,"POLYGON Z ((6.60946 52.15346 0.00000, 6.60837 ..."


In [5]:
# Here we sort the index:
catchment_boundaries = catchment_boundaries.set_index("id", inplace=False).sort_index(axis=0)
catchment_boundaries

,area_km2,outlet_lat,outlet_lng,area_offic,area_calc,Code,basin_id,label_area,name,layer,path,geometry
id,,,,,,,,,,,,
AT00001,4668.378643009853,47.274,9.535,4647.9,4668.379,AT00001,AT00001,0,Bangs,None,None,"POLYGON Z ((9.69406 46.54322 0.00000, 9.69570 ..."
AT00002,102.28688127787842,47.080,9.914,102,102.287,AT00002,AT00002,0,Schruns (Vonbunweg),None,None,"POLYGON Z ((10.13650 47.02949 0.00000, 10.1349..."
AT00003,536.2985313168554,47.133,9.848,535.2,536.299,AT00003,AT00003,0,Loruens-Aeule,None,None,"POLYGON Z ((10.11095 46.89437 0.00000, 10.1122..."
AT00004,66.2856923321305,47.129,10.062,66.6,66.286,AT00004,AT00004,0,Kloesterle (OEBB),None,None,"POLYGON Z ((10.14189 47.09706 0.00000, 10.1404..."
AT00005,72.4477297164234,47.151,9.803,72.2,72.448,AT00005,AT00005,0,Buers (Bruecke L82),None,None,"POLYGON Z ((9.67851 47.06249 0.00000, 9.67888 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR017,325,44.692,33.895,321,325.370,UAGR017,UAGR017,0,6682300,None,None,"POLYGON Z ((33.96791 44.63291 0.00000, 33.9679..."
UAGR018,48,44.500,34.167,49.7,47.594,UAGR018,UAGR018,0,6682500,None,None,"POLYGON Z ((34.19958 44.58291 0.00000, 34.2029..."
UAGR019,245,44.888,34.200,261,244.731,UAGR019,UAGR019,1,6683010,None,None,"POLYGON Z ((34.19624 44.88375 0.00000, 34.1962..."


## Daily streamflow time-series
It is important to note that this time-series was already filtered under a quality-check to delete potential outliers and negative values. 

In [6]:
timeseries_EU = pd.read_csv("data/streamflow/estreams_timeseries_streamflow.csv", index_col=0)
timeseries_EU.index = pd.to_datetime(timeseries_EU.index)
timeseries_EU.index.name = "dates"
timeseries_EU

,AT00001,AT00002,AT00003,AT00004,AT00005,AT00006,AT00007,AT00008,AT00009,AT00010,...,DE02101,DE02102,DE02103,DE02104,DE02105,DE02106,DE02107,DE02108,DE02109,DE02110
dates,,,,,,,,,,,,,,,,,,,,,
1900-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Replace any negative value by np.nan:
timeseries_EU = timeseries_EU.applymap(lambda x: np.nan if x < 0 else x)

## Streamflow gauges network

In [8]:
network_EU = pd.read_excel("data/streamflow/estreams_gauging_stations_infos.xlsx")
network_EU

,new_code,code,name,river,area,elev,lon,lat,country,degimpact,...,calc_Perc2,error_are2,xcoord_3,ycoord_3,manual_del,area_calc,area_calc_,area_calc_label,region,remark
0,ITVEN406,270VE,Adige a Albaredo,Adige,11954.0,23.59,11.266291,45.317948,IT,NaN,...,NaN,NaN,NaN,NaN,NaN,11938.762,0.0,0,NaN,NaN
1,ITVEN407,283VE,Brenta a Barzizza,Brenta,1567.0,105.38,11.730543,45.777415,IT,NaN,...,NaN,NaN,NaN,NaN,NaN,1571.329,0.0,0,NaN,NaN
2,ITVEN408,284VE,Adige a Boara Pisani,Adige,11954.0,8.4,11.784244,45.105481,IT,NaN,...,NaN,NaN,NaN,NaN,NaN,12020.860,0.0,0,NaN,NaN
3,ITVEN409,285VE,Tesina a Bolzano Vicentino,Tesina,694.0,37.57,11.620895,45.600894,IT,NaN,...,NaN,NaN,NaN,NaN,NaN,759.659,0.0,0,NaN,NaN
4,ITVEN410,286VE,Frassine a Borgofrassine,Frassine,NaN,16.69,11.481112,45.251409,IT,NaN,...,NaN,NaN,NaN,NaN,NaN,414.418,2.0,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15041,DE02106,5752400000,Hachelbich,Wipper,524.0,NaN,10.973000,51.347000,DE,NaN,...,NaN,NaN,NaN,NaN,NaN,524.000,0.0,0,TH,NaN
15042,DE02107,5752500000,Bleicherode,Bode,104.0,NaN,10.593000,51.450000,DE,NaN,...,NaN,NaN,NaN,NaN,NaN,104.000,0.0,0,TH,NaN
15043,DE02108,5754000000,Sundhausen,Helme,200.6,NaN,10.810000,51.468000,DE,NaN,...,NaN,NaN,NaN,NaN,NaN,200.000,0.0,0,TH,NaN
15044,DE02109,5755000000,Nordhausen,Zorge,303.6,NaN,10.784000,51.507000,DE,NaN,...,NaN,NaN,NaN,NaN,NaN,302.000,0.0,0,TH,NaN


In [9]:
network_EU.columns

Index(['new_code', 'code', 'name', 'river', 'area', 'elev', 'lon', 'lat',
       'country', 'degimpact', 'lake', 'num_reserv', 'RE', 'num_daily_',
       'num_years_', 'delete', 'calc_Perc_', 'xcoord', 'ycoord', 'xcoord_4',
       'ycoord_4', 'calc_area_', 'new_area', 'error_area', 'xcoord_2',
       'ycoord_2', 'new_area2', 'calc_Perc2', 'error_are2', 'xcoord_3',
       'ycoord_3', 'manual_del', 'area_calc', 'area_calc_', 'area_calc_label',
       'region', 'remark'],
      dtype='object')

In [10]:
network_EU[["new_code", "code"]]

,new_code,code
0,ITVEN406,270VE
1,ITVEN407,283VE
2,ITVEN408,284VE
3,ITVEN409,285VE
4,ITVEN410,286VE
...,...,...
15041,DE02106,5752400000
15042,DE02107,5752500000
15043,DE02108,5754000000
15044,DE02109,5755000000


In [11]:
network_EU_net = network_EU[["new_code", "code", "name", "river", "xcoord_4", "ycoord_4", "lon", "lat",
                           "area", "elev", "country", "area_calc", "area_calc_label"]]

network_EU_net.columns = ["basin_id", "gauge_id", "gauge_name ", "river", "lon", "lat", "lon_r","lat_r", 
                          "area_r", "elevation", "country", "area_c", "area_flag"]

network_EU_net

,basin_id,gauge_id,gauge_name,river,lon,lat,lon_r,lat_r,area_r,elevation,country,area_c,area_flag
0,ITVEN406,270VE,Adige a Albaredo,Adige,11.266291,45.317948,11.266291,45.317948,11954.0,23.59,IT,11938.762,0
1,ITVEN407,283VE,Brenta a Barzizza,Brenta,11.730543,45.777415,11.730543,45.777415,1567.0,105.38,IT,1571.329,0
2,ITVEN408,284VE,Adige a Boara Pisani,Adige,11.784244,45.105481,11.784244,45.105481,11954.0,8.4,IT,12020.860,0
3,ITVEN409,285VE,Tesina a Bolzano Vicentino,Tesina,11.620895,45.600894,11.620895,45.600894,694.0,37.57,IT,759.659,0
4,ITVEN410,286VE,Frassine a Borgofrassine,Frassine,11.481112,45.251409,11.481112,45.251409,NaN,16.69,IT,414.418,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15041,DE02106,5752400000,Hachelbich,Wipper,10.973000,51.347000,10.973000,51.347000,524.0,NaN,DE,524.000,0
15042,DE02107,5752500000,Bleicherode,Bode,10.593000,51.450000,10.593000,51.450000,104.0,NaN,DE,104.000,0
15043,DE02108,5754000000,Sundhausen,Helme,10.810000,51.468000,10.810000,51.468000,200.6,NaN,DE,200.000,0
15044,DE02109,5755000000,Nordhausen,Zorge,10.784000,51.507000,10.784000,51.507000,303.6,NaN,DE,302.000,0


In [12]:
network_EU_net["area_perc"] = ((network_EU_net["area_r"] - network_EU_net["area_c"])/network_EU_net["area_r"])  * 100

In [13]:
network_EU_net

,basin_id,gauge_id,gauge_name,river,lon,lat,lon_r,lat_r,area_r,elevation,country,area_c,area_flag,area_perc
0,ITVEN406,270VE,Adige a Albaredo,Adige,11.266291,45.317948,11.266291,45.317948,11954.0,23.59,IT,11938.762,0,0.127472
1,ITVEN407,283VE,Brenta a Barzizza,Brenta,11.730543,45.777415,11.730543,45.777415,1567.0,105.38,IT,1571.329,0,-0.276260
2,ITVEN408,284VE,Adige a Boara Pisani,Adige,11.784244,45.105481,11.784244,45.105481,11954.0,8.4,IT,12020.860,0,-0.559311
3,ITVEN409,285VE,Tesina a Bolzano Vicentino,Tesina,11.620895,45.600894,11.620895,45.600894,694.0,37.57,IT,759.659,0,-9.460951
4,ITVEN410,286VE,Frassine a Borgofrassine,Frassine,11.481112,45.251409,11.481112,45.251409,NaN,16.69,IT,414.418,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15041,DE02106,5752400000,Hachelbich,Wipper,10.973000,51.347000,10.973000,51.347000,524.0,NaN,DE,524.000,0,0.000000
15042,DE02107,5752500000,Bleicherode,Bode,10.593000,51.450000,10.593000,51.450000,104.0,NaN,DE,104.000,0,0.000000
15043,DE02108,5754000000,Sundhausen,Helme,10.810000,51.468000,10.810000,51.468000,200.6,NaN,DE,200.000,0,0.299103
15044,DE02109,5755000000,Nordhausen,Zorge,10.784000,51.507000,10.784000,51.507000,303.6,NaN,DE,302.000,0,0.527009


In [14]:
# Here we set the index:
network_EU_net.set_index("basin_id", inplace =True)
network_EU_net

,gauge_id,gauge_name,river,lon,lat,lon_r,lat_r,area_r,elevation,country,area_c,area_flag,area_perc
basin_id,,,,,,,,,,,,,
ITVEN406,270VE,Adige a Albaredo,Adige,11.266291,45.317948,11.266291,45.317948,11954.0,23.59,IT,11938.762,0,0.127472
ITVEN407,283VE,Brenta a Barzizza,Brenta,11.730543,45.777415,11.730543,45.777415,1567.0,105.38,IT,1571.329,0,-0.276260
ITVEN408,284VE,Adige a Boara Pisani,Adige,11.784244,45.105481,11.784244,45.105481,11954.0,8.4,IT,12020.860,0,-0.559311
ITVEN409,285VE,Tesina a Bolzano Vicentino,Tesina,11.620895,45.600894,11.620895,45.600894,694.0,37.57,IT,759.659,0,-9.460951
ITVEN410,286VE,Frassine a Borgofrassine,Frassine,11.481112,45.251409,11.481112,45.251409,NaN,16.69,IT,414.418,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE02106,5752400000,Hachelbich,Wipper,10.973000,51.347000,10.973000,51.347000,524.0,NaN,DE,524.000,0,0.000000
DE02107,5752500000,Bleicherode,Bode,10.593000,51.450000,10.593000,51.450000,104.0,NaN,DE,104.000,0,0.000000
DE02108,5754000000,Sundhausen,Helme,10.810000,51.468000,10.810000,51.468000,200.6,NaN,DE,200.000,0,0.299103


In [15]:
network_EU_net["start_date"] = find_first_non_nan_dates(timeseries_EU)
network_EU_net["end_date"] = find_last_non_nan_dates(timeseries_EU)

network_EU_net

,gauge_id,gauge_name,river,lon,lat,lon_r,lat_r,area_r,elevation,country,area_c,area_flag,area_perc,start_date,end_date
basin_id,,,,,,,,,,,,,,,
ITVEN406,270VE,Adige a Albaredo,Adige,11.266291,45.317948,11.266291,45.317948,11954.0,23.59,IT,11938.762,0,0.127472,2005-07-07,2022-12-31
ITVEN407,283VE,Brenta a Barzizza,Brenta,11.730543,45.777415,11.730543,45.777415,1567.0,105.38,IT,1571.329,0,-0.276260,2004-01-01,2022-12-31
ITVEN408,284VE,Adige a Boara Pisani,Adige,11.784244,45.105481,11.784244,45.105481,11954.0,8.4,IT,12020.860,0,-0.559311,2004-01-01,2022-12-31
ITVEN409,285VE,Tesina a Bolzano Vicentino,Tesina,11.620895,45.600894,11.620895,45.600894,694.0,37.57,IT,759.659,0,-9.460951,2004-01-01,2022-12-31
ITVEN410,286VE,Frassine a Borgofrassine,Frassine,11.481112,45.251409,11.481112,45.251409,NaN,16.69,IT,414.418,2,NaN,2012-11-11,2020-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE02106,5752400000,Hachelbich,Wipper,10.973000,51.347000,10.973000,51.347000,524.0,NaN,DE,524.000,0,0.000000,1961-11-01,2021-12-31
DE02107,5752500000,Bleicherode,Bode,10.593000,51.450000,10.593000,51.450000,104.0,NaN,DE,104.000,0,0.000000,1951-11-01,2021-12-31
DE02108,5754000000,Sundhausen,Helme,10.810000,51.468000,10.810000,51.468000,200.6,NaN,DE,200.000,0,0.299103,1957-11-01,2021-12-31


In [16]:
# Number of measurements:
network_EU_net[["num_daily_obs", "num_monthly", "num_monthly_complete", "num_yearly", "num_yearly_complete"]] = count_num_measurements(timeseries = timeseries_EU)
network_EU_net.drop(["num_monthly_complete", "num_yearly_complete"], axis = 1, inplace = True)
network_EU_net

,gauge_id,gauge_name,river,lon,lat,lon_r,lat_r,area_r,elevation,country,area_c,area_flag,area_perc,start_date,end_date,num_daily_obs,num_monthly,num_yearly
basin_id,,,,,,,,,,,,,,,,,,
ITVEN406,270VE,Adige a Albaredo,Adige,11.266291,45.317948,11.266291,45.317948,11954.0,23.59,IT,11938.762,0,0.127472,2005-07-07,2022-12-31,6157,204,18
ITVEN407,283VE,Brenta a Barzizza,Brenta,11.730543,45.777415,11.730543,45.777415,1567.0,105.38,IT,1571.329,0,-0.276260,2004-01-01,2022-12-31,6918,228,19
ITVEN408,284VE,Adige a Boara Pisani,Adige,11.784244,45.105481,11.784244,45.105481,11954.0,8.4,IT,12020.860,0,-0.559311,2004-01-01,2022-12-31,6918,228,19
ITVEN409,285VE,Tesina a Bolzano Vicentino,Tesina,11.620895,45.600894,11.620895,45.600894,694.0,37.57,IT,759.659,0,-9.460951,2004-01-01,2022-12-31,6918,228,19
ITVEN410,286VE,Frassine a Borgofrassine,Frassine,11.481112,45.251409,11.481112,45.251409,NaN,16.69,IT,414.418,2,NaN,2012-11-11,2020-12-31,2945,98,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE02106,5752400000,Hachelbich,Wipper,10.973000,51.347000,10.973000,51.347000,524.0,NaN,DE,524.000,0,0.000000,1961-11-01,2021-12-31,21976,722,61
DE02107,5752500000,Bleicherode,Bode,10.593000,51.450000,10.593000,51.450000,104.0,NaN,DE,104.000,0,0.000000,1951-11-01,2021-12-31,25629,842,71
DE02108,5754000000,Sundhausen,Helme,10.810000,51.468000,10.810000,51.468000,200.6,NaN,DE,200.000,0,0.299103,1957-11-01,2021-12-31,23437,770,65


In [17]:
# Calculate the number of gaps:
network_EU_net["num_days_gaps"] = ((network_EU_net['end_date'] - network_EU_net['start_date']).dt.days + 1) - network_EU_net['num_daily_obs']
network_EU_net

,gauge_id,gauge_name,river,lon,lat,lon_r,lat_r,area_r,elevation,country,area_c,area_flag,area_perc,start_date,end_date,num_daily_obs,num_monthly,num_yearly,num_days_gaps
basin_id,,,,,,,,,,,,,,,,,,,
ITVEN406,270VE,Adige a Albaredo,Adige,11.266291,45.317948,11.266291,45.317948,11954.0,23.59,IT,11938.762,0,0.127472,2005-07-07,2022-12-31,6157,204,18,230.0
ITVEN407,283VE,Brenta a Barzizza,Brenta,11.730543,45.777415,11.730543,45.777415,1567.0,105.38,IT,1571.329,0,-0.276260,2004-01-01,2022-12-31,6918,228,19,22.0
ITVEN408,284VE,Adige a Boara Pisani,Adige,11.784244,45.105481,11.784244,45.105481,11954.0,8.4,IT,12020.860,0,-0.559311,2004-01-01,2022-12-31,6918,228,19,22.0
ITVEN409,285VE,Tesina a Bolzano Vicentino,Tesina,11.620895,45.600894,11.620895,45.600894,694.0,37.57,IT,759.659,0,-9.460951,2004-01-01,2022-12-31,6918,228,19,22.0
ITVEN410,286VE,Frassine a Borgofrassine,Frassine,11.481112,45.251409,11.481112,45.251409,NaN,16.69,IT,414.418,2,NaN,2012-11-11,2020-12-31,2945,98,9,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE02106,5752400000,Hachelbich,Wipper,10.973000,51.347000,10.973000,51.347000,524.0,NaN,DE,524.000,0,0.000000,1961-11-01,2021-12-31,21976,722,61,0.0
DE02107,5752500000,Bleicherode,Bode,10.593000,51.450000,10.593000,51.450000,104.0,NaN,DE,104.000,0,0.000000,1951-11-01,2021-12-31,25629,842,71,0.0
DE02108,5754000000,Sundhausen,Helme,10.810000,51.468000,10.810000,51.468000,200.6,NaN,DE,200.000,0,0.299103,1957-11-01,2021-12-31,23437,770,65,0.0


In [18]:
# Here we compute the longest period with continuous measurements:
longest_gap_periods = longest_continuous_period(timeseries = timeseries_EU)

100%|█████████████████████████████████████| 15046/15046 [02:45<00:00, 91.07it/s]


In [19]:
network_EU_net[["num_continuous_days"]] = longest_gap_periods

In [20]:
# Here we sort the index:
network_EU_net = network_EU_net.sort_index(axis=0)
network_EU_net

,gauge_id,gauge_name,river,lon,lat,lon_r,lat_r,area_r,elevation,country,area_c,area_flag,area_perc,start_date,end_date,num_daily_obs,num_monthly,num_yearly,num_days_gaps,num_continuous_days
basin_id,,,,,,,,,,,,,,,,,,,,
AT00001,200014,Bangs,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,420,AT,4668.379,0,-0.440608,1996-01-01,2019-12-31,8766,288,24,0.0,8766
AT00002,200048,Schruns (Vonbunweg),Litz,9.913677,47.080301,9.913677,47.080301,102.0,673,AT,102.287,0,-0.281373,1958-10-01,2019-12-31,22372,735,62,0.0,22372
AT00003,231662,Loruens-Aeule,Ill,9.847765,47.132821,9.847765,47.132821,535.2,579,AT,536.299,0,-0.205344,1985-01-02,2019-12-31,12782,420,35,0.0,12782
AT00004,200592,Kloesterle (OEBB),Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,1014,AT,66.286,0,0.471471,1998-01-02,2019-12-31,8034,264,22,0.0,8034
AT00005,200097,Buers (Bruecke L82),Alvier,9.802668,47.150770,9.802668,47.150770,72.2,564,AT,72.448,0,-0.343490,1990-01-01,2019-12-31,10957,360,30,0.0,10957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR017,6682300,BASHTANOVKA,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,NaN,UA,325.370,0,-1.361371,1978-01-01,1987-12-31,3652,120,10,0.0,3652
UAGR018,6682500,YALTA,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,16,UA,47.594,0,4.237425,1978-01-01,1987-12-31,3652,120,10,0.0,3652
UAGR019,6683010,PIONERSKOE,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,307,UA,244.731,1,6.233333,1978-01-01,1987-12-31,3652,120,10,0.0,3652


In [21]:
network_EU_net.columns

Index(['gauge_id', 'gauge_name ', 'river', 'lon', 'lat', 'lon_r', 'lat_r',
       'area_r', 'elevation', 'country', 'area_c', 'area_flag', 'area_perc',
       'start_date', 'end_date', 'num_daily_obs', 'num_monthly', 'num_yearly',
       'num_days_gaps', 'num_continuous_days'],
      dtype='object')

In [23]:
network_EU_net.columns = ['gauge_id', 'gauge_name ', 'river', 'lon', 'lat', 'lon_r', 'lat_r',
                          'area_r', 'elevation', 'gauge_country', 'area_c', 'area_flag', 'area_perc',
                          'start_date', 'end_date', 'num_days', 'num_months', 'num_years',
                          'num_days_gaps', 'num_continuous_days']

network_EU_net = network_EU_net[['gauge_id', 'gauge_name ', 'gauge_country', 'river', 'lon', 'lat', 'lon_r', 'lat_r',
                          'area_r', 'elevation', 'area_c', 'area_flag', 'area_perc',
                          'start_date', 'end_date', 'num_years', 'num_months', 'num_days',
                          'num_days_gaps', 'num_continuous_days']]

network_EU_net

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,area_c,area_flag,area_perc,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days
basin_id,,,,,,,,,,,,,,,,,,,,
AT00001,200014,Bangs,AT,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,420,4668.379,0,-0.440608,1996-01-01,2019-12-31,24,288,8766,0.0,8766
AT00002,200048,Schruns (Vonbunweg),AT,Litz,9.913677,47.080301,9.913677,47.080301,102.0,673,102.287,0,-0.281373,1958-10-01,2019-12-31,62,735,22372,0.0,22372
AT00003,231662,Loruens-Aeule,AT,Ill,9.847765,47.132821,9.847765,47.132821,535.2,579,536.299,0,-0.205344,1985-01-02,2019-12-31,35,420,12782,0.0,12782
AT00004,200592,Kloesterle (OEBB),AT,Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,1014,66.286,0,0.471471,1998-01-02,2019-12-31,22,264,8034,0.0,8034
AT00005,200097,Buers (Bruecke L82),AT,Alvier,9.802668,47.150770,9.802668,47.150770,72.2,564,72.448,0,-0.343490,1990-01-01,2019-12-31,30,360,10957,0.0,10957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR017,6682300,BASHTANOVKA,UA,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,NaN,325.370,0,-1.361371,1978-01-01,1987-12-31,10,120,3652,0.0,3652
UAGR018,6682500,YALTA,UA,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,16,47.594,0,4.237425,1978-01-01,1987-12-31,10,120,3652,0.0,3652
UAGR019,6683010,PIONERSKOE,UA,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,307,244.731,1,6.233333,1978-01-01,1987-12-31,10,120,3652,0.0,3652


## New codes' definition:

In [24]:
# Adjust of the German names (inclusion of the region)
network_EU_net["region"] = network_EU.set_index("new_code", inplace = False)["region"]
network_EU_net[network_EU_net.gauge_country == "DE"]

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,area_flag,area_perc,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days,region
basin_id,,,,,,,,,,,,,,,,,,,,,
DE00001,5891200,"Alt Ruppin, Schleuse OP",DE,Rhin,12.835157,52.959469,12.835157,52.959469,534.785653,NaN,...,3,-134.602591,1980-11-01,2023-09-25,44,515,15669,0.0,15669,BB
DE00002,5871600,Babelsberg-Drewitz,DE,Nuthe,13.116304,52.362773,13.116304,52.362773,1768.326729,NaN,...,0,-1.485940,1954-11-01,2023-09-24,70,827,25160,5.0,25124,BB
DE00003,5530500,Bad Liebenwerda,DE,Schwarze Elster,13.394420,51.519043,13.394420,51.519043,3137.028384,NaN,...,3,26.821510,1960-11-01,2023-09-18,64,755,22967,0.0,22967,BB
DE00004,5930500,Bad Wilsnack,DE,Karthane,11.937284,52.956437,11.937284,52.956437,284.617208,NaN,...,0,-1.609808,1975-11-01,2023-09-24,47,539,16399,1096.0,14938,BB
DE00005,5825500,"Beeskow, Spreeschleuse UP",DE,Spree,14.254694,52.171976,14.254694,52.171976,5565.936643,NaN,...,0,3.436989,1950-03-02,2023-09-19,74,883,26865,0.0,26865,BB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE02106,5752400000,Hachelbich,DE,Wipper,10.973000,51.347000,10.973000,51.347000,524.000000,NaN,...,0,0.000000,1961-11-01,2021-12-31,61,722,21976,0.0,21976,TH
DE02107,5752500000,Bleicherode,DE,Bode,10.593000,51.450000,10.593000,51.450000,104.000000,NaN,...,0,0.000000,1951-11-01,2021-12-31,71,842,25629,0.0,25629,TH
DE02108,5754000000,Sundhausen,DE,Helme,10.810000,51.468000,10.810000,51.468000,200.600000,NaN,...,0,0.299103,1957-11-01,2021-12-31,65,770,23437,0.0,23437,TH


In [25]:
#create a old code column
network_EU_net['old_code'] = network_EU_net.index

# Extract the first four digits from the 'old_code' column
network_EU_net['first_four_digits'] = network_EU_net['old_code'].str.extract('([A-Za-z]{1,4})')

# Add zeros to ensure a total length of 4 characters
network_EU_net['first_four_digits'] = network_EU_net['first_four_digits'].apply(lambda x: x.ljust(4, '0'))

# Extract the numeric part from the 'code' column
network_EU_net['numeric_part'] = network_EU_net.old_code.str.extract('(\d+)')

# Convert 'numeric_part' to a 4-digit string with leading zeros
network_EU_net['numeric_part'] = network_EU_net['numeric_part'].apply(lambda x: f"{int(x):04d}")

## Create a new column 'new_code' by concatenating 'region' and 'numeric_part'
network_EU_net['new_code'] = network_EU_net['first_four_digits'] + network_EU_net['numeric_part']

network_EU_net

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,num_years,num_months,num_days,num_days_gaps,num_continuous_days,region,old_code,first_four_digits,numeric_part,new_code
basin_id,,,,,,,,,,,,,,,,,,,,,
AT00001,200014,Bangs,AT,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,420,...,24,288,8766,0.0,8766,NaN,AT00001,AT00,0001,AT000001
AT00002,200048,Schruns (Vonbunweg),AT,Litz,9.913677,47.080301,9.913677,47.080301,102.0,673,...,62,735,22372,0.0,22372,NaN,AT00002,AT00,0002,AT000002
AT00003,231662,Loruens-Aeule,AT,Ill,9.847765,47.132821,9.847765,47.132821,535.2,579,...,35,420,12782,0.0,12782,NaN,AT00003,AT00,0003,AT000003
AT00004,200592,Kloesterle (OEBB),AT,Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,1014,...,22,264,8034,0.0,8034,NaN,AT00004,AT00,0004,AT000004
AT00005,200097,Buers (Bruecke L82),AT,Alvier,9.802668,47.150770,9.802668,47.150770,72.2,564,...,30,360,10957,0.0,10957,NaN,AT00005,AT00,0005,AT000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR017,6682300,BASHTANOVKA,UA,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,NaN,...,10,120,3652,0.0,3652,NaN,UAGR017,UAGR,0017,UAGR0017
UAGR018,6682500,YALTA,UA,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,16,...,10,120,3652,0.0,3652,NaN,UAGR018,UAGR,0018,UAGR0018
UAGR019,6683010,PIONERSKOE,UA,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,307,...,10,120,3652,0.0,3652,NaN,UAGR019,UAGR,0019,UAGR0019


In [26]:
# Here we can check it:
network_EU_net[network_EU_net.gauge_country == "IT"]

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,num_years,num_months,num_days,num_days_gaps,num_continuous_days,region,old_code,first_four_digits,numeric_part,new_code
basin_id,,,,,,,,,,,,,,,,,,,,,
ITABR441,Aterno a Molina,Aterno a Molina,IT,Aterno,13.753056,42.137222,13.753056,42.137222,1303.0,454,...,10,114,3454,8.0,1954,NaN,ITABR441,ITAB,0441,ITAB0441
ITABR442,Aterno a Tre Ponti,Aterno a Tre Ponti,IT,Aterno,13.258056,42.476389,13.258056,42.476389,114.0,774,...,10,113,3250,212.0,620,NaN,ITABR442,ITAB,0442,ITAB0442
ITABR443,Feltrino a San Vito,Feltrino a San Vito,IT,Feltrino,14.439444,42.296389,14.439444,42.296389,50.0,24,...,10,114,3411,51.0,937,NaN,ITABR443,ITAB,0443,ITAB0443
ITABR444,Pescara a Maraone,Pescara a Maraone,IT,Pescara,13.838611,42.180000,13.838611,42.180000,2003.0,246,...,10,114,3455,7.0,1723,NaN,ITABR444,ITAB,0444,ITAB0444
ITABR445,Sagittario a Capo Canale,Sagittario a Capo Canale,IT,Sagittario,13.866389,42.123333,13.866389,42.123333,599.0,275,...,10,114,3456,6.0,2189,NaN,ITABR445,ITAB,0445,ITAB0445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ITVEN436,271VE,Piave a Belluno Ponte Vittoria,IT,Piave,12.218586,46.134222,12.218586,46.134222,NaN,NaN,...,3,30,826,134.0,422,NaN,ITVEN436,ITVE,0436,ITVE0436
ITVEN437,277VE,Cordevole a Ponte Mas,IT,Cordevole,12.125220,46.154313,12.125220,46.154313,696.0,360.75,...,21,213,6032,1308.0,2121,NaN,ITVEN437,ITVE,0437,ITVE0437
ITVEN438,327VE,Piave a Segusino,IT,Piave,11.948837,45.922490,11.948837,45.922490,3469.0,177.39,...,19,216,6284,656.0,2041,NaN,ITVEN438,ITVE,0438,ITVE0438


* ### Germany

In [27]:
# Subset only germany
network_EU_net_DE = network_EU_net[network_EU_net.gauge_country == "DE"]

# Extract the numeric part from the 'code' column
network_EU_net_DE['numeric_part'] = network_EU_net_DE.old_code.str.extract('(\d+)')

# Convert 'numeric_part' to a 4-digit string with leading zeros
network_EU_net_DE['numeric_part'] = network_EU_net_DE['numeric_part'].apply(lambda x: f"{int(x):04d}")

## Create a new column 'new_code' by concatenating 'region' and 'numeric_part'
network_EU_net_DE['new_code'] = "DE" + network_EU_net_DE['region'] + network_EU_net_DE['numeric_part']

network_EU_net_DE

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,num_years,num_months,num_days,num_days_gaps,num_continuous_days,region,old_code,first_four_digits,numeric_part,new_code
basin_id,,,,,,,,,,,,,,,,,,,,,
DE00001,5891200,"Alt Ruppin, Schleuse OP",DE,Rhin,12.835157,52.959469,12.835157,52.959469,534.785653,NaN,...,44,515,15669,0.0,15669,BB,DE00001,DE00,0001,DEBB0001
DE00002,5871600,Babelsberg-Drewitz,DE,Nuthe,13.116304,52.362773,13.116304,52.362773,1768.326729,NaN,...,70,827,25160,5.0,25124,BB,DE00002,DE00,0002,DEBB0002
DE00003,5530500,Bad Liebenwerda,DE,Schwarze Elster,13.394420,51.519043,13.394420,51.519043,3137.028384,NaN,...,64,755,22967,0.0,22967,BB,DE00003,DE00,0003,DEBB0003
DE00004,5930500,Bad Wilsnack,DE,Karthane,11.937284,52.956437,11.937284,52.956437,284.617208,NaN,...,47,539,16399,1096.0,14938,BB,DE00004,DE00,0004,DEBB0004
DE00005,5825500,"Beeskow, Spreeschleuse UP",DE,Spree,14.254694,52.171976,14.254694,52.171976,5565.936643,NaN,...,74,883,26865,0.0,26865,BB,DE00005,DE00,0005,DEBB0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE02106,5752400000,Hachelbich,DE,Wipper,10.973000,51.347000,10.973000,51.347000,524.000000,NaN,...,61,722,21976,0.0,21976,TH,DE02106,DE00,2106,DETH2106
DE02107,5752500000,Bleicherode,DE,Bode,10.593000,51.450000,10.593000,51.450000,104.000000,NaN,...,71,842,25629,0.0,25629,TH,DE02107,DE00,2107,DETH2107
DE02108,5754000000,Sundhausen,DE,Helme,10.810000,51.468000,10.810000,51.468000,200.600000,NaN,...,65,770,23437,0.0,23437,TH,DE02108,DE00,2108,DETH2108


In [28]:
# here we replace only for Germany and check the results:
network_EU_net.loc[network_EU_net_DE.index, "new_code"] = network_EU_net_DE['new_code']
network_EU_net[network_EU_net.gauge_country == "DE"]

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,num_years,num_months,num_days,num_days_gaps,num_continuous_days,region,old_code,first_four_digits,numeric_part,new_code
basin_id,,,,,,,,,,,,,,,,,,,,,
DE00001,5891200,"Alt Ruppin, Schleuse OP",DE,Rhin,12.835157,52.959469,12.835157,52.959469,534.785653,NaN,...,44,515,15669,0.0,15669,BB,DE00001,DE00,0001,DEBB0001
DE00002,5871600,Babelsberg-Drewitz,DE,Nuthe,13.116304,52.362773,13.116304,52.362773,1768.326729,NaN,...,70,827,25160,5.0,25124,BB,DE00002,DE00,0002,DEBB0002
DE00003,5530500,Bad Liebenwerda,DE,Schwarze Elster,13.394420,51.519043,13.394420,51.519043,3137.028384,NaN,...,64,755,22967,0.0,22967,BB,DE00003,DE00,0003,DEBB0003
DE00004,5930500,Bad Wilsnack,DE,Karthane,11.937284,52.956437,11.937284,52.956437,284.617208,NaN,...,47,539,16399,1096.0,14938,BB,DE00004,DE00,0004,DEBB0004
DE00005,5825500,"Beeskow, Spreeschleuse UP",DE,Spree,14.254694,52.171976,14.254694,52.171976,5565.936643,NaN,...,74,883,26865,0.0,26865,BB,DE00005,DE00,0005,DEBB0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE02106,5752400000,Hachelbich,DE,Wipper,10.973000,51.347000,10.973000,51.347000,524.000000,NaN,...,61,722,21976,0.0,21976,TH,DE02106,DE00,2106,DETH2106
DE02107,5752500000,Bleicherode,DE,Bode,10.593000,51.450000,10.593000,51.450000,104.000000,NaN,...,71,842,25629,0.0,25629,TH,DE02107,DE00,2107,DETH2107
DE02108,5754000000,Sundhausen,DE,Helme,10.810000,51.468000,10.810000,51.468000,200.600000,NaN,...,65,770,23437,0.0,23437,TH,DE02108,DE00,2108,DETH2108


* ### Belgium

In [29]:
# Subset only Belgium
network_EU_net_BE = network_EU_net[network_EU_net.gauge_country == "BE"]

## Create a new column 'new_code' by concatenating 'region' and 'numeric_part'
network_EU_net_BE['new_code'] = "BE" + network_EU_net_BE['region'] + network_EU_net_BE['numeric_part']

network_EU_net_BE

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,num_years,num_months,num_days,num_days_gaps,num_continuous_days,region,old_code,first_four_digits,numeric_part,new_code
basin_id,,,,,,,,,,,,,,,,,,,,,
BE00001,10461002,Bierges,BE,Dyle,4.606129,50.710583,4.606129,50.710583,310.0,NaN,...,47,564,17162,5.0,15955,WA,BE00001,BE00,0001,BEWA0001
BE00002,19511002,Tubize,BE,Senne,4.196446,50.690256,4.196446,50.690256,215.0,NaN,...,24,288,8766,0.0,8766,WA,BE00002,BE00,0002,BEWA0002
BE00003,23711002,Ronquieres,BE,Samme,4.224809,50.605923,4.224809,50.605923,135.0,NaN,...,33,396,12053,0.0,12053,WA,BE00003,BE00,0003,BEWA0003
BE00004,29521002,Irchonwelz,BE,DendreOrientale,3.751661,50.618903,3.751661,50.618903,126.0,NaN,...,44,528,16071,0.0,16071,WA,BE00004,BE00,0004,BEWA0004
BE00005,29711002,Ath,BE,DendreOrientale,3.777834,50.633521,3.777834,50.633521,229.0,NaN,...,45,540,16436,0.0,16436,WA,BE00005,BE00,0005,BEWA0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BE00226,01L07_287,Ternat,BE,Steenvoordebeek,4.176000,50.876000,4.176000,50.876000,NaN,NaN,...,40,454,13456,1314.0,3758,VL,BE00226,BE00,0226,BEVL0226
BE00227,01L07_289,Viane,BE,Mark,3.892000,50.741000,3.892000,50.741000,NaN,NaN,...,48,556,16430,700.0,1905,VL,BE00227,BE00,0227,BEVL0227
BE00228,01L01_493,Vlamertinge,BE,Grote Kemmelbeek,2.811000,50.844000,2.811000,50.844000,NaN,NaN,...,41,475,13350,2416.0,9860,VL,BE00228,BE00,0228,BEVL0228


In [30]:
# here we replace only for Germany and check the results:
network_EU_net.loc[network_EU_net_BE.index, "new_code"] = network_EU_net_BE['new_code']
network_EU_net[network_EU_net.gauge_country == "BE"]

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,num_years,num_months,num_days,num_days_gaps,num_continuous_days,region,old_code,first_four_digits,numeric_part,new_code
basin_id,,,,,,,,,,,,,,,,,,,,,
BE00001,10461002,Bierges,BE,Dyle,4.606129,50.710583,4.606129,50.710583,310.0,NaN,...,47,564,17162,5.0,15955,WA,BE00001,BE00,0001,BEWA0001
BE00002,19511002,Tubize,BE,Senne,4.196446,50.690256,4.196446,50.690256,215.0,NaN,...,24,288,8766,0.0,8766,WA,BE00002,BE00,0002,BEWA0002
BE00003,23711002,Ronquieres,BE,Samme,4.224809,50.605923,4.224809,50.605923,135.0,NaN,...,33,396,12053,0.0,12053,WA,BE00003,BE00,0003,BEWA0003
BE00004,29521002,Irchonwelz,BE,DendreOrientale,3.751661,50.618903,3.751661,50.618903,126.0,NaN,...,44,528,16071,0.0,16071,WA,BE00004,BE00,0004,BEWA0004
BE00005,29711002,Ath,BE,DendreOrientale,3.777834,50.633521,3.777834,50.633521,229.0,NaN,...,45,540,16436,0.0,16436,WA,BE00005,BE00,0005,BEWA0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BE00226,01L07_287,Ternat,BE,Steenvoordebeek,4.176000,50.876000,4.176000,50.876000,NaN,NaN,...,40,454,13456,1314.0,3758,VL,BE00226,BE00,0226,BEVL0226
BE00227,01L07_289,Viane,BE,Mark,3.892000,50.741000,3.892000,50.741000,NaN,NaN,...,48,556,16430,700.0,1905,VL,BE00227,BE00,0227,BEVL0227
BE00228,01L01_493,Vlamertinge,BE,Grote Kemmelbeek,2.811000,50.844000,2.811000,50.844000,NaN,NaN,...,41,475,13350,2416.0,9860,VL,BE00228,BE00,0228,BEVL0228


* ## Final adjusts

In [31]:
network_EU_net.drop(["region", "first_four_digits", "numeric_part"], axis = 1, inplace = True)
network_EU_net

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,area_perc,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days,old_code,new_code
basin_id,,,,,,,,,,,,,,,,,,,,,
AT00001,200014,Bangs,AT,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,420,...,-0.440608,1996-01-01,2019-12-31,24,288,8766,0.0,8766,AT00001,AT000001
AT00002,200048,Schruns (Vonbunweg),AT,Litz,9.913677,47.080301,9.913677,47.080301,102.0,673,...,-0.281373,1958-10-01,2019-12-31,62,735,22372,0.0,22372,AT00002,AT000002
AT00003,231662,Loruens-Aeule,AT,Ill,9.847765,47.132821,9.847765,47.132821,535.2,579,...,-0.205344,1985-01-02,2019-12-31,35,420,12782,0.0,12782,AT00003,AT000003
AT00004,200592,Kloesterle (OEBB),AT,Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,1014,...,0.471471,1998-01-02,2019-12-31,22,264,8034,0.0,8034,AT00004,AT000004
AT00005,200097,Buers (Bruecke L82),AT,Alvier,9.802668,47.150770,9.802668,47.150770,72.2,564,...,-0.343490,1990-01-01,2019-12-31,30,360,10957,0.0,10957,AT00005,AT000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR017,6682300,BASHTANOVKA,UA,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,NaN,...,-1.361371,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR017,UAGR0017
UAGR018,6682500,YALTA,UA,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,16,...,4.237425,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR018,UAGR0018
UAGR019,6683010,PIONERSKOE,UA,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,307,...,6.233333,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR019,UAGR0019


## Add data provider

In [32]:
# Read the catalogue
estreams_streamflow_catalogue = pd.read_excel("data/streamflow/estreams_streamflow_catalogue.xlsx")
estreams_streamflow_catalogue

,provider_id,code_basins,provider_country,country_code,provider_name,license,platform,num_stations,boundaries,start_date,end_date,source_license,source_streamflow,source_gauges_infos,source_boundaries,Observations
0,IT_ISPRA,ITIS,ITALY,IT,Institute for Environmental Protection and Res...,-,Website,NaN,Not available,NaN,NaN,NaN,http://www.hiscentral.isprambiente.gov.it/hisc...,http://www.hiscentral.isprambiente.gov.it/hisc...,-,NaN
1,ES_CEDEX,ES,SPAIN,ES,Centro de Estudios Hidrograficos (CEDEX),-,Website,NaN,Not available,NaN,NaN,NaN,https://www.ceh.cedex.es/anuarioaforos/demarca...,https://www.ceh.cedex.es/anuarioaforos/demarca...,-,NaN
2,PT_SNIRH,PT,PORTUGAL,PT,Sistema nacional de Informacao de Recursos Hid...,-,Website,NaN,Not available,NaN,NaN,NaN,https://snirh.apambiente.pt,https://snirh.apambiente.pt,NaN,NaN
3,CH_CAMELS,CH,SWITZERLAND,CH,Catchment Attributes and MEteorology for large...,CC-BY-4.0,Website,NaN,Not available,NaN,NaN,NaN,https://zenodo.org/doi/10.5281/zenodo.7784632,https://zenodo.org/doi/10.5281/zenodo.7784632,https://zenodo.org/doi/10.5281/zenodo.7784632,NaN
4,GB_NRFA,GB,GREAT BRITAIN,GB,National River flow Archive (NRFA) – UK Centre...,No-redistribution,Website,NaN,Available,NaN,NaN,NaN,https://nrfa.ceh.ac.uk/data/search#collapse-do...,https://nrfa.ceh.ac.uk/data/search#collapse-do...,https://catalogue.ceh.ac.uk/documents/8344e4f3...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,DE_NW,DENW,GERMANY,DE,"Ministerium fur Umwelt, Naturschutz und Verkeh...",DL-DE-BY,Website,NaN,Not available,NaN,NaN,NaN,https://www.elwasweb.nrw.de/elwas-web/data-obj...,NaN,NaN,NaN
63,DE_NI,DENI,GERMANY,DE,Niedersachsischer Landesbetrieb fur Wasserwirt...,DL-DE-BY,Website,NaN,Not available,NaN,NaN,NaN,http://www.wasserdaten.niedersachsen.de/cadenz...,http://www.wasserdaten.niedersachsen.de/cadenz...,NaN,NaN
64,DE_SN,DESN,GERMANY,DE,Landeschochwasserzentrum. Datenportal fur Umwe...,NaN,Website,NaN,Not available,NaN,NaN,NaN,https://www.umwelt.sachsen.de/umwelt/infosyste...,NaN,NaN,NaN
65,DE_ST,DEST,GERMANY,DE,Landesbetrieb fur Hochwasserschutz und Wasserw...,Redsitribution allowed,Website,NaN,Not available,NaN,NaN,NaN,https://gld.lhw-sachsen-anhalt.de/,NaN,NaN,NaN


In [33]:
# Assign the values for each gauge
network_EU_net["gauge_provider"] = np.nan

for code, provider_id in tqdm(zip(estreams_streamflow_catalogue.code_basins, estreams_streamflow_catalogue.provider_id)):
    network_EU_net_code = network_EU_net[network_EU_net.new_code.str.contains(code, case=False)]
    network_EU_net.loc[network_EU_net_code.index, "gauge_provider"] = provider_id

67it [00:00, 277.10it/s]


In [34]:
network_EU_net

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days,old_code,new_code,gauge_provider
basin_id,,,,,,,,,,,,,,,,,,,,,
AT00001,200014,Bangs,AT,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,420,...,1996-01-01,2019-12-31,24,288,8766,0.0,8766,AT00001,AT000001,AT_EHYD
AT00002,200048,Schruns (Vonbunweg),AT,Litz,9.913677,47.080301,9.913677,47.080301,102.0,673,...,1958-10-01,2019-12-31,62,735,22372,0.0,22372,AT00002,AT000002,AT_EHYD
AT00003,231662,Loruens-Aeule,AT,Ill,9.847765,47.132821,9.847765,47.132821,535.2,579,...,1985-01-02,2019-12-31,35,420,12782,0.0,12782,AT00003,AT000003,AT_EHYD
AT00004,200592,Kloesterle (OEBB),AT,Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,1014,...,1998-01-02,2019-12-31,22,264,8034,0.0,8034,AT00004,AT000004,AT_EHYD
AT00005,200097,Buers (Bruecke L82),AT,Alvier,9.802668,47.150770,9.802668,47.150770,72.2,564,...,1990-01-01,2019-12-31,30,360,10957,0.0,10957,AT00005,AT000005,AT_EHYD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR017,6682300,BASHTANOVKA,UA,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,NaN,...,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR017,UAGR0017,UA_GRDC
UAGR018,6682500,YALTA,UA,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,16,...,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR018,UAGR0018,UA_GRDC
UAGR019,6683010,PIONERSKOE,UA,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,307,...,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR019,UAGR0019,UA_GRDC


In [35]:
# Check the data
network_EU_net[network_EU_net.gauge_provider.isna()]

,gauge_id,gauge_name,gauge_country,river,lon,lat,lon_r,lat_r,area_r,elevation,...,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days,old_code,new_code,gauge_provider
basin_id,,,,,,,,,,,,,,,,,,,,,
DE01945,2730010,BundespegelKoeln,DE,Rhine,6.963,50.937,6.963,50.937,144232.0,NaN,...,1900-01-02,2021-07-31,121,1441,43859,547.0,43828,DE01945,DEBU1945,NaN
DE01946,2750010,BundespegelDuesseldorf,DE,Rhine,6.770,51.226,6.770,51.226,147680.0,NaN,...,1901-01-01,2021-07-31,120,1429,43495,547.0,43464,DE01946,DEBU1946,NaN
DE01947,2730010,BundespegelKoeln,DE,Rhine,6.963,50.937,6.963,50.937,144232.0,NaN,...,1900-01-02,2021-07-31,121,1441,43859,547.0,43828,DE01947,DEBU1947,NaN
DE01948,2750010,BundespegelDuesseldorf,DE,Rhine,6.770,51.226,6.770,51.226,147680.0,NaN,...,1901-01-01,2021-07-31,120,1429,43495,547.0,43464,DE01948,DEBU1948,NaN
DE01949,2790010,BundespegelRees,DE,Rhine,6.395,51.757,6.395,51.757,159300.0,NaN,...,1900-01-02,2021-07-31,121,1441,43859,547.0,43828,DE01949,DEBU1949,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DE02106,5752400000,Hachelbich,DE,Wipper,10.973,51.347,10.973,51.347,524.0,NaN,...,1961-11-01,2021-12-31,61,722,21976,0.0,21976,DE02106,DETH2106,NaN
DE02107,5752500000,Bleicherode,DE,Bode,10.593,51.450,10.593,51.450,104.0,NaN,...,1951-11-01,2021-12-31,71,842,25629,0.0,25629,DE02107,DETH2107,NaN
DE02108,5754000000,Sundhausen,DE,Helme,10.810,51.468,10.810,51.468,200.6,NaN,...,1957-11-01,2021-12-31,65,770,23437,0.0,23437,DE02108,DETH2108,NaN


In [36]:
network_EU_net.columns

Index(['gauge_id', 'gauge_name ', 'gauge_country', 'river', 'lon', 'lat',
       'lon_r', 'lat_r', 'area_r', 'elevation', 'area_c', 'area_flag',
       'area_perc', 'start_date', 'end_date', 'num_years', 'num_months',
       'num_days', 'num_days_gaps', 'num_continuous_days', 'old_code',
       'new_code', 'gauge_provider'],
      dtype='object')

## Organize the columns

In [37]:
network_EU_net.columns = ['gauge_id', 'gauge_name ', 'gauge_country', 'river', 'lon_snap', 'lat_snap',
                          'lon', 'lat', 'area', 'elevation', 'area_calc', 'area_flag',
                          'area_perc', 'start_date', 'end_date', 'num_years', 'num_months',
                          'num_days', 'num_days_gaps', 'num_continuous_days', 'old_code', 'new_code', 'gauge_provider']

network_EU_net = network_EU_net[['gauge_id', 'gauge_name ', 'gauge_country', 'gauge_provider', 'river', 'lon_snap',
                                 'lat_snap', 'lon', 'lat', 'area', 'elevation', 
                                 'area_calc', 'area_flag', 'area_perc',
                                 'start_date', 'end_date', 'num_years', 'num_months', 'num_days',
                                 'num_days_gaps', 'num_continuous_days', 'new_code']]

network_EU_net

,gauge_id,gauge_name,gauge_country,gauge_provider,river,lon_snap,lat_snap,lon,lat,area,...,area_flag,area_perc,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days,new_code
basin_id,,,,,,,,,,,,,,,,,,,,,
AT00001,200014,Bangs,AT,AT_EHYD,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,...,0,-0.440608,1996-01-01,2019-12-31,24,288,8766,0.0,8766,AT000001
AT00002,200048,Schruns (Vonbunweg),AT,AT_EHYD,Litz,9.913677,47.080301,9.913677,47.080301,102.0,...,0,-0.281373,1958-10-01,2019-12-31,62,735,22372,0.0,22372,AT000002
AT00003,231662,Loruens-Aeule,AT,AT_EHYD,Ill,9.847765,47.132821,9.847765,47.132821,535.2,...,0,-0.205344,1985-01-02,2019-12-31,35,420,12782,0.0,12782,AT000003
AT00004,200592,Kloesterle (OEBB),AT,AT_EHYD,Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,...,0,0.471471,1998-01-02,2019-12-31,22,264,8034,0.0,8034,AT000004
AT00005,200097,Buers (Bruecke L82),AT,AT_EHYD,Alvier,9.802668,47.150770,9.802668,47.150770,72.2,...,0,-0.343490,1990-01-01,2019-12-31,30,360,10957,0.0,10957,AT000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR017,6682300,BASHTANOVKA,UA,UA_GRDC,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,...,0,-1.361371,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR0017
UAGR018,6682500,YALTA,UA,UA_GRDC,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,...,0,4.237425,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR0018
UAGR019,6683010,PIONERSKOE,UA,UA_GRDC,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,...,1,6.233333,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR0019


In [38]:
# Subset the data for only catchments with valid areas:
timeseries_EU = timeseries_EU.loc[:, catchment_boundaries.index]
network_EU_net = network_EU_net.loc[catchment_boundaries.index, :]
network_EU_net

,gauge_id,gauge_name,gauge_country,gauge_provider,river,lon_snap,lat_snap,lon,lat,area,...,area_flag,area_perc,start_date,end_date,num_years,num_months,num_days,num_days_gaps,num_continuous_days,new_code
id,,,,,,,,,,,,,,,,,,,,,
AT00001,200014,Bangs,AT,AT_EHYD,Rhein,9.534835,47.273748,9.534835,47.273748,4647.9,...,0,-0.440608,1996-01-01,2019-12-31,24,288,8766,0.0,8766,AT000001
AT00002,200048,Schruns (Vonbunweg),AT,AT_EHYD,Litz,9.913677,47.080301,9.913677,47.080301,102.0,...,0,-0.281373,1958-10-01,2019-12-31,62,735,22372,0.0,22372,AT000002
AT00003,231662,Loruens-Aeule,AT,AT_EHYD,Ill,9.847765,47.132821,9.847765,47.132821,535.2,...,0,-0.205344,1985-01-02,2019-12-31,35,420,12782,0.0,12782,AT000003
AT00004,200592,Kloesterle (OEBB),AT,AT_EHYD,Alfenz,10.061843,47.128994,10.061843,47.128994,66.6,...,0,0.471471,1998-01-02,2019-12-31,22,264,8034,0.0,8034,AT000004
AT00005,200097,Buers (Bruecke L82),AT,AT_EHYD,Alvier,9.802668,47.150770,9.802668,47.150770,72.2,...,0,-0.343490,1990-01-01,2019-12-31,30,360,10957,0.0,10957,AT000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UAGR017,6682300,BASHTANOVKA,UA,UA_GRDC,KACHA,33.894739,44.691884,33.900000,44.683333,321.0,...,0,-1.361371,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR0017
UAGR018,6682500,YALTA,UA,UA_GRDC,DERE-KIOY,34.166667,44.500000,34.166667,44.500000,49.7,...,0,4.237425,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR0018
UAGR019,6683010,PIONERSKOE,UA,UA_GRDC,SALHYR,34.199841,44.887685,34.200000,44.883333,261.0,...,1,6.233333,1978-01-01,1987-12-31,10,120,3652,0.0,3652,UAGR0019


In [39]:
# Save the new data:
network_EU_net.to_excel('results/estreams_gauging_stations.xlsx')
timeseries_EU.to_csv('results/estreams_timeseries_streamflow.csv')

## End